Connecting to Google Drive 

In [ ]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
import import_ipynb 
connect_to_drive()

#The path of the root folder bioinformatics should be inserted here in order to import all the modules
root_folder='/content/gdrive/My Drive/Bioinformatics3/bioinformatics/'
import sys
sys.path.append(root_folder)

Mounted at /content/gdrive


In [ ]:
#Put this to False if you don't intend to use the wandb library to log the results (suggested)
use_wandb=False
save_model=False

Importing modules

In [ ]:
import torch
from utils.utils import *
install_requirements(use_wandb)
import albumentations as A
from torchvision import transforms
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.augmentations.transforms import *
import segmentation_models_pytorch as smp
from torchmetrics.functional import iou as iou_torch
from tqdm import tqdm
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR
from torchvision.ops.focal_loss import sigmoid_focal_loss
from segmentation_models_pytorch.losses import FocalLoss

if use_wandb:
  import wandb


from Dataset.VascularDataset import *

Retrieve images paths from txt

In [ ]:
input_path_training='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Train'
input_path_test='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Test'

if save_model is True:
  save_model_folder='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'

txt_with_training_paths_inputs=root_folder+'Inputs/training_paths_inputs.txt'
txt_with_training_paths_targets=root_folder+'Inputs/training_paths_targets.txt'
txt_with_validation_paths_inputs=root_folder+'Inputs/validation_paths_inputs.txt'
txt_with_validation_paths_targets=root_folder+'Inputs/validation_paths_targets.txt'
txt_with_test_paths_inputs=root_folder+'Inputs/test_paths_inputs.txt'
txt_with_test_paths_targets=root_folder+'Inputs/test_paths_targets.txt'

imgs_training_input_split=read_list_images_path_from_txt(txt_with_training_paths_inputs)
imgs_training_target_split=read_list_images_path_from_txt(txt_with_training_paths_targets)
imgs_validation_input_split=read_list_images_path_from_txt(txt_with_validation_paths_inputs)
imgs_validation_target_split=read_list_images_path_from_txt(txt_with_validation_paths_targets)
imgs_test_input=read_list_images_path_from_txt(txt_with_test_paths_inputs)
imgs_test_target=read_list_images_path_from_txt(txt_with_test_paths_targets)

Defining transformations

In [ ]:
affine_transform=A.Compose([Flip(), ShiftScaleRotate()])
elastic_transform=A.Compose([ElasticTransform()])
pixel_wise_transform=A.Compose([GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
affine_elastic_pixelwise_transform=A.Compose([Flip(), ShiftScaleRotate(), ElasticTransform(), GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
slice_length=1000
resize_transform=A.Compose([RandomCrop(slice_length, slice_length)])

Defining model, loss function, metrics, and initializing Wandb

In [ ]:
model_name='Unet'
batch_size=10
lr=0.3
momentum=0.9
max_n_epochs=80
encoder_name='timm-resnest101e'
dataset_type='Standard'
step_size=8
gamma=0.9
transforms=affine_elastic_pixelwise_transform

model = smp.Unet(
    encoder_name=encoder_name,       
    encoder_weights='imagenet', 
    in_channels=3,
    activation='sigmoid',                 
)

def dice_loss(pred,target):
    numerator = 2 * torch.sum(pred * target)
    denominator = torch.sum(pred + target)
    return 1 - (numerator + 1) / (denominator + 1)

model.cuda()
optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
loss=dice_loss
scheduler=StepLR(optimizer, step_size=step_size, gamma=gamma)

#To use weighted loss:
#criterion=BCEWithLogitsLoss(torch.Tensor([2.3]).cuda())

if use_wandb:
  wandb.init(project="Bioinformatics_segmentation_experiments_Unet++",
            config={
                "batch_size": batch_size,
                "model": model_name,
                "learning_rate": lr,
                "encoder_name": encoder_name,
                "max_n_epochs": max_n_epochs,
                "momentum": momentum,
                "optimizer": optimizer,
                "dataset": dataset_type,
                "loss": loss,
                "metrics": iou_torch,
                "transforms": transforms,
                "scheduler": scheduler,
                "step_size":step_size,
                "gamma":gamma
            })
  run_id=wandb.run.id
  
if save_model:
  save_model_path=save_model_folder+run_id

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-resnest/resnest101-22405ba7.pth" to /root/.cache/torch/hub/checkpoints/resnest101-22405ba7.pth


  0%|          | 0.00/185M [00:00<?, ?B/s]

Defining dataset and dataloaders

In [ ]:
training_dataset=VascularDataset(imgs_training_input_split, imgs_training_target_split, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685), transform=transforms)
validation_dataset=VascularDataset(imgs_validation_input_split, imgs_validation_target_split, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685))
test_dataset=VascularDataset(imgs_test_input, imgs_test_target)

print(f'Training dataset length: {len(training_dataset)}')
print(f'Validation dataset length: {len(validation_dataset)}')
print(f'Test dataset length: {len(test_dataset)}')

train_dataloader=DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_dataloader=DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

Training dataset length: 160
Validation dataset length: 40
Test dataset length: 10


Training, validation and logging into Wandb

In [15]:
lowest_validation_loss=float('inf')

for epoch in range(1, max_n_epochs+1, 1):

  model.train()
  training_loss=0
  training_iou=0
  training_samples=0

  for x,y in tqdm(train_dataloader): 
    optimizer.zero_grad()
    out=model(x.cuda().float())
    myloss=loss(out, y.cuda().float())

    for out_i,y_i in zip(out,y.cuda()):
      iou=iou_torch(out_i.detach(),y_i.int())
      training_iou+=iou
      training_samples+=1

    myloss.backward()
    optimizer.step()
    training_loss+=myloss.item()
  
  print(f'Epoch #{epoch}, training loss: {training_loss/training_samples}, training iou: {training_iou/training_samples}')

  scheduler.step()

  model.eval()
  validation_loss=0
  validation_iou=0
  validation_samples=0

  for x,y in tqdm(valid_dataloader): 
    out=model(x.cuda())
    myloss=loss(out.float(), y.cuda().float())

    for out_i,y_i in zip(out,y.cuda()):
      iou=iou_torch(out_i.detach(),y_i.int())
      validation_iou+=iou
      validation_samples+=1

    validation_loss+=myloss.item()

  print(f'Epoch #{epoch}, validation loss: {validation_loss/validation_samples}, validation iou: {validation_iou/validation_samples}')

  if validation_loss/validation_samples<lowest_validation_loss:
    lowest_validation_loss=validation_loss/validation_samples
    last_epoch_saved=epoch
    if save_model:
      torch.save(model,f'{save_model_path}.pth')

  if use_wandb:
    wandb.log({"training loss": training_loss/training_samples, "validation loss": validation_loss/validation_samples, "epoch":epoch})
    wandb.log({"training iou": training_iou/training_samples, "validation iou": validation_iou/validation_samples, "epoch": epoch})

print(f'Last epoch saved: {last_epoch_saved}')

if use_wandb:
  wandb.log({"last epoch saved": last_epoch_saved})

100%|██████████| 16/16 [01:01<00:00,  3.87s/it]


Epoch #1, training loss: 0.03942755721509457, training iou: 0.5092045068740845


100%|██████████| 4/4 [00:11<00:00,  2.87s/it]


Epoch #1, validation loss: 0.0371509850025177, validation iou: 0.47939711809158325


100%|██████████| 16/16 [01:04<00:00,  4.05s/it]


...

Epoch #4, validation loss: 0.031650510430335996, validation iou: 0.5884682536125183


100%|██████████| 16/16 [00:58<00:00,  3.68s/it]


Epoch #5, training loss: 0.03542803041636944, training iou: 0.5650064945220947


  0%|          | 0/4 [00:03<?, ?it/s]


KeyboardInterrupt: ignored